# 데이터 준비

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
from glob import glob
import math
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!ls '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/5. 해커톤/1차 해커톤/data'

디즈니	아마존	파라마운트  넷플릭스


In [4]:
data_dir = '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/5. 해커톤/1차 해커톤/processed_data4'
save_data_dir = '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/5. 해커톤/1차 해커톤/processed_data5'
platform_names = ["amazon", "netflix", "disney", "paramount"]

credits_path = os.path.join(data_dir, "merged_credits.csv")
amazon_titles_path = os.path.join(data_dir, "merged_titles.csv")

In [5]:
credits = pd.read_csv(credits_path)
titles = pd.read_csv(amazon_titles_path)

# 다작 감독 분석

In [6]:
credits_director = credits.loc[credits["role"] == "DIRECTOR"]
credits_actor = credits.loc[credits["role"] == "ACTOR"]

In [7]:
credits_director

,person_id,id,name,character,role,platform
25,28732,tm19248,Clyde Bruckman,NaN,DIRECTOR,amazon
26,21174,tm19248,Buster Keaton,NaN,DIRECTOR,amazon
57,13717,tm82253,William Wyler,NaN,DIRECTOR,amazon
93,23839,tm83884,Howard Hawks,NaN,DIRECTOR,amazon
113,35387,tm56584,Nicholas Ray,NaN,DIRECTOR,amazon
...,...,...,...,...,...,...
267696,1767751,tm994534,Safyah Usmani,NaN,DIRECTOR,paramount
267697,357110,tm1045408,Kate Davis,NaN,DIRECTOR,paramount
267698,68620,tm1045408,David Heilbroner,NaN,DIRECTOR,paramount
267700,2078564,tm1106415,Alex Gale,NaN,DIRECTOR,paramount


In [8]:
# 상위 30위의 감독
rank_threshold = 100

In [9]:
# top_rank_director_id = credits_director["person_id"].value_counts().tolist()
top_rank_director_id = credits_director["person_id"].value_counts().index.tolist()
top_rank_director_id = top_rank_director_id[:rank_threshold]
# top_rank_director_id

- 영화 중 탑인 것

In [10]:
blank_df = pd.DataFrame(data=[], columns=titles.columns)

id_list = []
ex_id_len = 0
for director_id in top_rank_director_id:
    ids = credits.loc[credits["person_id"] == director_id]
    ids = ids["id"].tolist()
    id_list.append(ex_id_len + len(ids))
    ex_id_len = ex_id_len + len(ids)
    for id in ids:
        titles_movie = titles.loc[titles["type"] == "MOVIE"]
        titles_movie = titles_movie.loc[titles_movie["id"] == id]
        blank_df = pd.concat([blank_df, titles_movie])

In [11]:
# id_list

In [12]:
titles["release_year"][-1:]

17376    2021
Name: release_year, dtype: int64

# 상위 100 영화의 기간 별 imdb 점수 그래프

In [13]:
plt.figure(figsize=(10, 500))
ex_id = 0
for i, id in enumerate(id_list):
    a = blank_df[ex_id:id]
    a = a.sort_values("release_year", ascending=True)
    year = a["release_year"].tolist()
    plt.subplot(100, 1, i+1)
    plt.plot(range(id-ex_id), a["imdb_score"])

    plt.title("rank({0}) / released years({1}-{2})".format(i, year[:1], year[-1:]))
    ex_id = id
plt.show()

Output hidden; open in https://colab.research.google.com to view.

#### 다작인 감독의 투표 수

In [14]:
# 상위 30위의 감독
rank_threshold = 100

In [15]:
# top_rank_director_id = credits_director["person_id"].value_counts().tolist()
# 다작 감독 아이디 리스트 생성
top_rank_director_id = credits_director["person_id"].value_counts().index.tolist()
top_rank_director_id = top_rank_director_id[:rank_threshold]
# top_rank_director_id

In [16]:
blank_df = pd.DataFrame(data=[], columns=titles.columns)

id_list = []
ex_id_len = 0
for director_id in top_rank_director_id:
    # 해당 감독의 작품 id 리스트 생성
    ids = credits.loc[credits["person_id"] == director_id]
    ids = ids["id"].tolist()
    id_list.append(ex_id_len + len(ids))
    ex_id_len = ex_id_len + len(ids)
    # 작품 중 영화인 것만 고른 후 영화 id에 맞는 행을 모음
    for id in ids:
        titles_movie = titles.loc[titles["type"] == "MOVIE"]
        titles_movie = titles_movie.loc[titles_movie["id"] == id]
        blank_df = pd.concat([blank_df, titles_movie])

- 스코어/인기도/투표수 같이 보여주고
- 해당 영화 제목도 같이 보여주자

In [17]:
plt.figure(figsize=(10, 500))
ex_id = 0
for i, id in enumerate(id_list):
    a = blank_df[ex_id:id]
    a = a.sort_values("release_year", ascending=True)
    year = a["release_year"].tolist()
    plt.subplot(100, 1, i+1)
    plt.plot(range(id-ex_id), a["imdb_votes"])
    plt.title("rank({0}) / released years({1}-{2})".format(i, year[:1], year[-1:]))
    ex_id = id
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [18]:
import math

In [19]:
def convert_log10(x):
    x = math.log10(x)

    return x

In [21]:
plt.figure(figsize=(10, 500))
ex_id = 0
for i, id in enumerate(id_list):
    a = blank_df[ex_id:id]
    a = a.sort_values("release_year", ascending=True)
    year = a["release_year"].tolist()
    pop = a["imdb_votes"].apply(convert_log10)
    pop = pop.tolist()

    ax0 = plt.subplot(100, 1, i+1)
    ax1 = ax0.twinx()

    # ax0.plot(range(id-ex_id), a["imdb_votes"], "r")
    c1 = ax0.plot(range(id-ex_id), pop, "r")
    ax0.set_ylabel("imdb_votes")
    ax0.set_ylim(0, 7)
    
    c2 = ax1.plot(range(id-ex_id), a["imdb_score"], "b")
    ax1.set_ylabel("imdb_score")
    ax1.set_ylim(0, 10)

    c = c1 + c2
    ax0.legend(c, ['imdb_votes', 'imdb_score'])

    plt.title("rank({0}) / released years({1}-{2})".format(i, year[:1], year[-1:]))
    ex_id = id
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# 다작이 아니어도 괜찮은가?